In [131]:
import pandas as pd
import numpy as np
import binance_socket
import dataDownloaders
import labelling_functions
import helper_functions
import imp
import configparser
import threading
from binance_socket import binancePublicSocket
import matplotlib.pyplot as plt
import statistics
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
import seaborn as sns
from matplotlib.pyplot import figure
import math
import forecasting_backtesting
import pickle as pk
from datetime import datetime,timedelta
from binance.client import Client
import pygsheets
import time

In [21]:
with open(r"/Users/Hugolu88/Documents/repos/binance_v2/strategies/btc_bnb_stat_arb/variables.pickle", "rb") as output_file:
    loaded = pk.load(output_file)

In [80]:
googlesheetbase = pygsheets.authorize(service_file='/Users/Hugolu88/Documents/repos/binance_v2/gsdata-313315-6c56edae3685.json')
sheet = googlesheetbase.open('Indicators')
worksheet= sheet[0]
start_row = 44
start_column = 4

In [31]:
pairs = loaded[0]
parameters = loaded[1]
constants = loaded[2]
betas = loaded[3]
signal_roll = loaded[4]
X2 = [x[1] for x in pairs]
X1 = [x[0] for x in pairs]
len_all_items = len(pairs)
all_items = list(pd.Series([x[0] for x in pairs]+[x[1] for x in pairs]).drop_duplicates())

In [41]:
config = configparser.ConfigParser()
config.read('/Users/Hugolu88/Documents/repos/binance_v2/config_global.ini')
key = config['PARAMETERS']['key']
scrt = config['PARAMETERS']['scrt']

print("Initiating Binance client")
float_list = ["Open","High","Low","Close","Volume","Quote asset volume","Number of trades","Taker buy base asset volume","Taker buy quote asset volume"]
storage = []
storage_lock = threading.Lock()
client = binancePublicSocket(storage,storage_lock,api_key = key,api_secret = scrt)

Initiating Binance client


In [23]:
#We also need the alphas, betas to construct the signal, that is, x2hat. The signal roll (to calculate the window)
# for the signal
# the granularity to get the data as well

In [141]:
helper = helper_functions.helper()
days_back = helper.input_float("days back (remember model in 15 minutes)")

end_string = datetime.now().strftime("%d %b, %Y")
start_string = (datetime.now() - timedelta(days=  days_back)).strftime("%d %b, %Y")
utc_time = lambda x: datetime.fromtimestamp(x/1000).strftime("%Y-%m-%d %H:%M:%S")

klines = []
for i in all_items:
    print("getting data for: "+str(i))
    klines.append(client.get_historical_klines(i, Client.KLINE_INTERVAL_15MINUTE, start_string,end_string))

initiating
Please input the days back (remember model in 15 minutes) (float)
40
getting data for: LINKUSDT
getting data for: LTCUSDT


In [142]:
data = []
for i1,i in enumerate(klines):
    rider =pd.DataFrame(i,columns = ["open_time","Open","High","Low","Close","Volume","Close time","Quote asset volume","Number of trades","Taker buy base asset volume","Taker buy quote asset volume","ignore"])
    rider['open_time'] = rider['open_time'].apply(utc_time)
    rider['Close time'] = rider['Close time'].apply(utc_time)
    rider['ticker'] = all_items[i1]
    for j in float_list:
        rider[j] = rider[j].astype("float")
    data.append(rider)
data_dictionary = dict(zip(all_items,data))



In [143]:
print("preparing pairs data")
pairs_data = []
for i,item in enumerate(pairs):
    x1 = data_dictionary[item[0]]['Close']
    x2 = data_dictionary[item[1]]['Close']
    time_= data_dictionary[item[0]]['open_time']
    signal_raw = (x1*betas[i]+constants[i]-x2)
    rolling_signal = (signal_raw-signal_raw.rolling(signal_roll).mean())/signal_raw.rolling(signal_roll).std()
    pairs_data.append(pd.DataFrame(np.array([x1,x2,signal_raw,rolling_signal,time_]).T,columns = [item[0],item[1],'signal_raw','signal','open_time']))

preparing pairs data


In [102]:
means = []
standard_deviations = []
for i,item in enumerate(pairs_data):
    means.append(item['signal_raw'].rolling(signal_roll).mean()[len(item)-1])
    standard_deviations.append(item['signal_raw'].rolling(signal_roll).std()[len(item)-1])



In [103]:
pump_dic = dict(zip(all_items,pairs_data))

In [147]:
#reminder of what we need:
pump_dic = pump_dic
parameters = parameters
constants = constants
betas = betas
means = means
standard_deviations = standard_deviations
X1= X1
X2 = X2
min_15_data= data_dictionary.copy()
pairs_data_copy = pairs_data.copy()

In [127]:
print("sending tickers to gsheet")
worksheet.set_dataframe(pd.DataFrame(pairs),(start_row,start_column-2))
print("sending params to gsheet")
worksheet.set_dataframe(pd.DataFrame(parameters),(start_row,start_column))

sending tickers to gsheet
sending params to gsheet


In [128]:
print("Preparing to loop")

Preparing to loop


In [169]:
iterations = 0
sleep_factor = 0.5
print("fetching data")



while True:
    config = configparser.ConfigParser()
    config.read('/Users/Hugolu88/Documents/repos/binance_v2/config_global.ini')
    switch = config['FILE_ON']['statarb']
    if switch != "Yes":
        break
    klines= []
    print("Calcing signals")
    #Signal calcing
    pricingdata = client.get_all_tickers()

    symbols = [x['symbol'] for x in pricingdata]
    prices = [x['price'] for x in pricingdata]
    price_dict = dict(zip(symbols,prices))
    relevant = [float(price_dict[x]) for x in all_items]

    for i,item in enumerate(pairs):
        upper,lower= parameters[i][0],parameters[i][1]
        signal = (float(price_dict[item[0]])*betas[i]+constants[0])-float(price_dict[item[1]])
        normalised_signal = (signal - means[i])/standard_deviations[i]
        worksheet.update_value((start_row+1+i,start_column+3),normalised_signal)
    #DATA DOWNLOAD LOOP START ----------------------------------------------------
    for i in all_items:

        while True: #Keep trying to append something to the historical klines until we can't
            try:
                klines.append(client.get_historical_klines(i, Client.KLINE_INTERVAL_15MINUTE, "1000 minutes ago UTC"))
                counter = 0
                break
            except:
                time.sleep(10)
                counter += 1
                if counter > 400:
                    print("ERROR TIME OUT")
                    break
    data = []

    for i1,item in enumerate(klines):
        rider =pd.DataFrame(item,columns = ["open_time","Open","High","Low","Close","Volume","Close time","Quote asset volume","Number of trades","Taker buy base asset volume","Taker buy quote asset volume","ignore"])
        rider['open_time'] = rider['open_time'].apply(utc_time)
        rider['Close time'] = rider['Close time'].apply(utc_time)
        rider['ticker'] = all_items[i1]
        for j in float_list:
            rider[j] = rider[j].astype("float")
        data.append(rider)
    for j,i in enumerate(data):
        appends = i[i['open_time']>min_15_data[all_items[j]]['open_time'].max()]
        min_15_data[all_items[j]] = min_15_data[all_items[j]].append(appends).reset_index(drop=True)
        
    for i,item in enumerate(pairs):
        x1 = min_15_data[item[0]]['Close']
        x2 = min_15_data[item[1]]['Close']
        time_= min_15_data[item[0]]['open_time']
        signal_raw = (x1*betas[i]+constants[i]-x2)
        rolling_signal = (signal_raw-signal_raw.rolling(signal_roll).mean())/signal_raw.rolling(signal_roll).std()
        rider = pd.DataFrame(np.array([x1,x2,signal_raw,rolling_signal,time_]).T,columns = [item[0],item[1],'signal_raw','signal','open_time'])
        appends = rider[rider['open_time']>pairs_data_copy[i]['open_time'].max()]
        pairs_data_copy[i] = pairs_data_copy[i].append(appends).reset_index(drop=True)
        means[i] = pairs_data_copy[i]['signal_raw'].rolling(signal_roll).mean()[len(pairs_data_copy[i])-1]
        standard_deviations[i] = pairs_data_copy[i]['signal_raw'].rolling(signal_roll).std()[len(pairs_data_copy[i])-1]               

    time.sleep(10) # sleep to avoid connection timeouts


fetching data
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals
Calcing signals


KeyboardInterrupt: 